In [32]:
import joblib
import pandas as pd
import sys, pathlib

# Setup ruta al proyecto
PROJECT_ROOT = pathlib.Path().resolve().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg

In [34]:
data = joblib.load(cfg.DATA / "processed" / "lstm5d_data.pkl")
df_prices = pd.read_parquet(cfg.DATA / "raw" / "prices.parquet")

X = data["X"]
y = data["y"]

# Filtrar muestras inválidas
import numpy as np
mask = ~np.isnan(y).any(axis=1) & ~np.isinf(y).any(axis=1)
X = X[mask]
y = y[mask]


print("X:", X.shape, "y:", y.shape)


X: (4450, 60, 80) y: (4450, 40)


In [76]:
n = len(X)
train_size = int(n * 0.7)
val_size = int(n * 0.15)

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

from sklearn.preprocessing import StandardScaler

# Escalar inputs (X)
X_all_2d = X.reshape(-1, X.shape[2])  # entrenamos con todo X, no solo con train
scaler_X = StandardScaler()
X_all_scaled = scaler_X.fit_transform(X_all_2d).reshape(X.shape)


X_train_scaled = X_all_scaled[:train_size]
X_val_scaled   = X_all_scaled[train_size:train_size + val_size]
X_test_scaled  = X_all_scaled[train_size + val_size:]


# Filtrar muestras inválidas en X (después de escalar)
mask_valid = ~np.isnan(X_train_scaled_raw).any(axis=(1, 2))
X_train_scaled = X_train_scaled_raw[mask_valid]
y_train_scaled = y_train[mask_valid]  # y sin escalar aún, se alinea aquí

# Ahora escala y_train
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train_scaled)

# Guardar escaladores
joblib.dump(scaler_X, cfg.MODELS / "scaler_X_lstm5d.pkl")
joblib.dump(scaler_y, cfg.MODELS / "scaler_y_lstm5d.pkl")

print("Train:", X_train.shape)
print("Val:", X_val.shape)
print("Test:", X_test.shape)

Train: (3115, 60, 80)
Val: (667, 60, 80)
Test: (668, 60, 80)


In [78]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(cfg.WINDOW, X.shape[2])),  # 60 x 80
    layers.LSTM(64, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(32),
    layers.Dense(y.shape[1])  # ← ✅ Solo 40 salidas (1 por activo)
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 60, 64)         │        37,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 60, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 40)             │         1,320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,856 (198.66 KB)

 Trainable params: 50,856 (198.66 KB)

 Non-trainable params: 0 (0.00 B)

In [80]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(patience=5, restore_best_weights=True)

print("Check X_train_scaled:", np.isnan(X_train_scaled).sum(), "NaNs /", np.isinf(X_train_scaled).sum(), "Infs")
print("Check y_train_scaled:", np.isnan(y_train_scaled).sum(), "NaNs /", np.isinf(y_train_scaled).sum(), "Infs")
print("Check X_val_scaled:", np.isnan(X_val_scaled).sum(), "NaNs /", np.isinf(X_val_scaled).sum(), "Infs")
print("Check y_val_scaled:", np.isnan(y_val_scaled).sum(), "NaNs /", np.isinf(y_val_scaled).sum(), "Infs")
print("X_train range:", np.min(X_train_scaled), "→", np.max(X_train_scaled))
print("y_train range:", np.min(y_train_scaled), "→", np.max(y_train_scaled))



history = model.fit(
 X_train_scaled, y_train_scaled,
 validation_data=(X_val_scaled, y_val_scaled),
 epochs=50,
 batch_size=32,
 callbacks=[early_stop],
 verbose=1
)



Check X_train_scaled: 0 NaNs / 0 Infs
Check y_train_scaled: 0 NaNs / 0 Infs
Check X_val_scaled: 0 NaNs / 0 Infs
Check y_val_scaled: 0 NaNs / 0 Infs
X_train range: -17.821685428605505 → 53.052622871642846
y_train range: -12.675652251513881 → 24.020961515649844
Epoch 1/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 13s 64ms/step - loss: 0.9977 - val_loss: 1.4488
Epoch 2/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - loss: 0.9491 - val_loss: 1.5133
Epoch 3/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - loss: 0.8675 - val_loss: 1.5347
Epoch 4/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - loss: 0.7987 - val_loss: 1.5504
Epoch 5/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - loss: 0.7415 - val_loss: 1.5955
Epoch 6/50
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.7144 - val_loss: 1.6002


In [82]:
from pathlib import Path

# Crear carpeta si no existe
Path("models").mkdir(parents=True, exist_ok=True)

model.save(cfg.MODELS / "lstm5d.keras")
print("✅ Modelo guardado en formato .keras")
joblib.dump(history.history, cfg.RESULT / "history_lstm5d.pkl")


✅ Modelo guardado en formato .keras


['C:\\Users\\ferra\\Documents\\TFM\\results\\history_lstm5d.pkl']

In [84]:
y_pred = model.predict(X_test_scaled)
rmse = np.sqrt(((y_test_scaled - y_pred)**2).mean(axis=0))
rmse_mean = rmse.mean()
print("RMSE medio:", rmse_mean)

joblib.dump(rmse_mean, cfg.RESULT / "rmse_lstm5d.pkl")


21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step
RMSE medio: 1.0346944027561495


['C:\\Users\\ferra\\Documents\\TFM\\results\\rmse_lstm5d.pkl']

El valor de RMSE medio obtenido (1.03) se justifica por la mayor complejidad del modelo LSTM al trabajar con ventanas de 60 días y 80 variables por muestra, lo cual introduce alta dimensionalidad y mayor varianza en los errores. Aunque el rendimiento es inferior al de otras configuraciones con menos variables (RMSE ≈ 0.14), este modelo captura dinámicas más ricas, lo que puede resultar valioso al combinarse con métodos evolutivos en la fase de optimización de carteras.